In [19]:

fn = lambda x: ('gaze' in str(x).lower())
srs = frvrs_logs_df.applymap(fn, na_action='ignore').sum()
columns_list = srs[srs != 0].index.tolist()
print(columns_list)
for cn in columns_list:
    print(cn)
    mask_series = frvrs_logs_df[cn].map(fn)
    df = frvrs_logs_df[mask_series]
    display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

['action_type', 'voice_capture_message']
action_type


,163464,160247,163912,162678
action_type,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE
elapsed_time,59006,145298,116037,164748
event_time,2023-09-11 13:46:04,2023-09-11 10:12:48,2023-09-11 13:47:01,2023-09-11 13:05:32
session_uuid,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,b11f14c2-8de9-4247-9753-1434cb392804,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17
file_name,d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv,b11f14c2-8de9-4247-9753-1434cb392804.csv,d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17.csv
logger_version,1.3,1.3,1.3,1.3
time_group,1,1,1,1
player_gaze_location,"(1.4, 0.0, 4.0)","(2.9, 0.0, 2.5)","(0.0, 0.0, 25.0)","(2.9, 0.0, 2.5)"
player_gaze_patient_id,Bob_7 Root,Tutorial Military Marine Root,Helga_14 Root,Tutorial Military Marine Root
player_gaze_distance_to_patient,1.421842,1.552028,1.252505,1.736698


voice_capture_message


,63648
action_type,VOICE_CAPTURE
elapsed_time,154228
event_time,2023-05-11 10:47:00
session_uuid,4226ad6c-bb0a-4df0-8c4a-64c312927578
file_name,clean-4226ad6c-bb0a-4df0-8c4a-64c312927578.csv
logger_version,1.0
time_group,1
voice_capture_message,there you go it's like every time he moves his...


In [ ]:


# tick_locations = ax.get_xticks()
# print(tick_locations)

# tick_labels = ax.get_xticklabels()
# print(tick_labels)